In [59]:
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timezone, timedelta

In [60]:
search = pd.read_pickle('search.pkl')
youtube = pd.read_pickle('youtube.pkl')

search['title'] = search['title'].apply(lambda x: f"S: {x}")
youtube['title'] = youtube['title'].apply(lambda x: f"YT: {x}")

In [61]:
search

,title,datetime
123,S: https://www.formistudio.app/113443/edit,2025-01-31 23:10:33.112601042+00:00
124,S: Dashboard,2025-01-31 23:10:26.627115965+00:00
125,S: Sign in - Google Accounts,2025-01-31 23:10:23.398943901+00:00
126,S: Sign in - Google Accounts,2025-01-31 23:10:19.575409889+00:00
127,S: https://www.formistudio.app/login,2025-01-31 23:10:17.006270885+00:00
...,...,...
1556,S: college gender ratio,2024-12-01 18:14:01.848561049+00:00
1557,S: college gender ratio,2024-12-01 07:54:04.206163883+00:00
1558,S: uva rage room,2024-12-01 07:53:54.028465986+00:00
1559,S: https://www.google.com/search#q=uva+rage+ro...,2024-12-01 00:12:20.593153954+00:00


In [62]:
youtube

,title,datetime
775,YT: UVA HooRaas | Raas All-Stars XIV - 2022 | ...,2025-01-31 17:48:45+00:00
776,YT: How to make 50k in one night 🤣🤣🤣#shorts #f...,2025-01-30 18:57:49+00:00
777,YT: Don’t click the sound,2025-01-30 18:57:48+00:00
778,"YT: Clash Royale Live Stream | Best Decks, Str...",2025-01-30 18:57:20+00:00
779,"YT: Unbeatable Bulls: 6 Finals, 6 Wins 🏆🔥#shorts",2025-01-30 18:57:06+00:00
...,...,...
5647,YT: Plot armor but it's Bruce Wayne's wealth...,2024-12-01 18:03:49+00:00
5648,YT: Genesis ￼GV80 | Paintless Dent Removal,2024-12-01 18:00:09+00:00
5649,YT: You owe me one more. #show #sports #clips,2024-12-01 17:58:38+00:00
5650,"YT: ""Rex's new weapon🐱‍🏍""#invincible #series #...",2024-12-01 17:57:27+00:00


In [63]:
complete_usage = pd.concat([search, youtube])

In [64]:
df = complete_usage.sort_values(by='datetime')
df['timedelta'] = df['datetime'].diff()
df["datetime"] = (df["datetime"] - timedelta(hours=5)).dt.tz_localize(None)

In [65]:
# Create a new DataFrame to store adjacent values
comparison_df = pd.DataFrame({
    'prev_search': df['title'].shift(1),  
    'next_search': df['title'],
    'Prev_Datetime': df['datetime'].shift(1),
    'Next_Datetime': df['datetime'],
    'Timedelta': df['timedelta']
}).reset_index(drop=True)
comparison_df['Prev_Hour'] = comparison_df['Prev_Datetime'].dt.hour
comparison_df['Next_Hour'] = comparison_df['Next_Datetime'].dt.hour
NIGHT_START = 20  # 8 PM
NIGHT_END = 5     # 5 AM
WAKE_START = 7    # 7 AM
WAKE_END = 16     # 4 PM

# Filter the comparison DataFrame based on the threshold
sleep_df = comparison_df[
    (comparison_df['Timedelta'] > pd.Timedelta(hours=5)) &  # Inactivity period
    (
        (comparison_df['Prev_Hour'] >= NIGHT_START) |  # After 8 PM
        (comparison_df['Prev_Hour'] <= NIGHT_END)      # Before 5 AM
    ) &
    (
        (comparison_df['Next_Hour'] >= WAKE_START) &   # After 7 AM
        (comparison_df['Next_Hour'] <= WAKE_END)       # Before 4 PM
    )
]
# Drop NaN values caused by shifting
sleep_df = sleep_df.dropna()

In [68]:
sleep_df['Timedelta'] = comparison_df['Timedelta'].apply(lambda x: x.total_seconds() / 3600)
sleep_df = sleep_df.drop(columns=["Prev_Hour", "Next_Hour"], errors='ignore')
print(len(sleep_df))
sleep_df

53


,prev_search,next_search,Prev_Datetime,Next_Datetime,Timedelta
4,S: college gender ratio,YT: You won’t guess how many knives he has on ...,2024-12-01 02:54:04.206163883,2024-12-01 12:56:20.000000000,10.037721
108,S: beanie on dude,YT: Bane Finally Stops Taking Venom and Honors...,2024-12-02 01:05:31.626600981,2024-12-02 13:23:04.000000000,12.292326
189,YT: Minecraft Word Wordle 3,S: NetBadge Message - Loading Session Information,2024-12-03 01:59:22.000000000,2024-12-03 10:24:47.847094059,8.423846
302,YT: UTD TaRaas | Raas All-Stars XV - 2023 | Ba...,YT: Roy and Ted chat about home affairs#movie ...,2024-12-03 23:19:07.000000000,2024-12-04 14:08:48.000000000,14.828056
404,YT: Russo and his father have different views ...,S: 2D Quadrotor PID Control,2024-12-05 01:25:04.000000000,2024-12-05 13:16:46.855195045,11.861904
551,YT: r/Tumblr - food secret agent,YT: The Most Famous Movie Bloopers that are Fu...,2024-12-06 00:15:46.000000000,2024-12-06 11:49:31.000000000,11.562500
819,YT: MrBeast Lost This Huge Lawsuit!,S: are.tapeworms.deadly,2024-12-07 02:06:42.000000000,2024-12-07 13:10:15.899624109,11.059417
899,YT: Why the hell did he dump me?#funny #tvshow...,S: karan das 1862,2024-12-08 01:30:48.000000000,2024-12-08 12:36:57.634705067,11.102676
1116,S: RAS XVII- AV Feedback Submission,YT: When Domino’s Pizza Entered Italy🇮🇹🍕,2024-12-09 00:14:22.814846039,2024-12-09 12:33:55.000000000,12.325607
1327,S: what is 150 degrees in radians,YT: SPLASHHHHHHHH,2024-12-11 05:56:54.571976900,2024-12-11 13:40:47.000000000,7.731230


# Sleep Statistics

In [67]:
sleep_df['Timedelta'].mean()

np.float64(10.928246118228513)